In [15]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pickle

In [17]:
# Load data_set
data=pd.read_csv("Churn_Modelling.csv")
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [18]:
# preprocess the data
# remove irrelevant columns
data=data.drop(['RowNumber','CustomerId','Surname'], axis=1)
data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [ ]:
## Encode categorical variables
## we will now encode gender as 1 and 0
label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])
data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,1,39,5,0.00,2,1,0,96270.64,0
9996,516,France,1,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,0,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,1,42,3,75075.31,2,1,0,92888.52,1


In [22]:
## One Hot Encode Geography column, because we cannot change all types of locations into 1s and 2s or 0s.
from sklearn.preprocessing import OneHotEncoder
onehot_encoder_geo=OneHotEncoder()
geo_encoder = onehot_encoder_geo.fit_transform(data[['Geography']])
geo_encoder

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 10000 stored elements and shape (10000, 3)>

In [23]:
onehot_encoder_geo.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [13]:
geo_encoded_df=pd.DataFrame(geo_encoder.toarray(), columns=onehot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [24]:
## combine one hot encoder columns with thw original data
data=pd.concat([data.drop('Geography', axis=1), geo_encoded_df], axis=1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [26]:
## save encoders and scaler (onehot_encoder_geo and label_encoder_gender)

with open('label_encoder_gender.pkl','wb') as file:
    pickle.dump(label_encoder_gender, file)
    
with open('onehot_encoder_geo.pkl','wb') as file:
    pickle.dump(onehot_encoder_geo, file)


In [27]:
## Divide the data set into independent and dependent features
x=data.drop('Exited', axis=1)
y=data['Exited']

## Split the data in training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.2, random_state=42)

## Scaling these features
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [ ]:
x_train

In [ ]:
x_test

In [30]:
with open('scaler.pkl','wb') as file:
    pickle.dump(scaler, file)

# ANN Implementaion

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

In [33]:
model = Sequential([
    Dense(64, activation='relu', input_shape=(x_train.shape[1],)), ## Hidden layer 1 connected with input layer
    Dense(32, activation='relu'), ## hidden layer 2
    Dense(1, activation='sigmoid') ## output layer
])

In [34]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                832       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [35]:
## Compile the model
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=['accuracy'])

In [37]:
## Setup tensorboard
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
log_dir="logs/fit"+ datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callbacks = TensorBoard(log_dir=log_dir, histogram_freq=1)


In [38]:
## set up early stopping
early_stopping_callbacks = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)


In [39]:
## Training the Model
history = model.fit(
    x_train, y_train, validation_data=(x_test, y_test), epochs=100,
    callbacks=[tensorflow_callbacks, early_stopping_callbacks]
)

Epoch 1/100


250/250 [==============================] - 10s 18ms/step - loss: 0.4634 - accuracy: 0.7995 - val_loss: 0.3954 - val_accuracy: 0.8265
Epoch 2/100
250/250 [==============================] - 2s 8ms/step - loss: 0.3898 - accuracy: 0.8399 - val_loss: 0.3593 - val_accuracy: 0.8540
Epoch 3/100
250/250 [==============================] - 2s 9ms/step - loss: 0.3571 - accuracy: 0.8547 - val_loss: 0.3488 - val_accuracy: 0.8570
Epoch 4/100
250/250 [==============================] - 2s 10ms/step - loss: 0.3464 - accuracy: 0.8561 - val_loss: 0.3431 - val_accuracy: 0.8590
Epoch 5/100
250/250 [==============================] - 3s 10ms/step - loss: 0.3406 - accuracy: 0.8581 - val_loss: 0.3435 - val_accuracy: 0.8640
Epoch 6/100
250/250 [==============================] - 2s 9ms/step - loss: 0.3374 - accuracy: 0.8586 - val_loss: 0.3433 - val_accuracy: 0.8585
Epoch 7/100
250/250 [==============================] - 2s 9ms/step - loss: 0.3338 - accuracy: 0.8631 - val_loss: 0.3412 - val_accuracy: 

In [40]:
model.save('model.h5')

d:\Gaurav\AI related\ANN CLASSIFICATION\venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [44]:
## load tensorflow extension
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [45]:
%tensorboard --logdir logs/fit20250827-161516

Reusing TensorBoard on port 6007 (pid 29676), started 0:03:11 ago. (Use '!kill 29676' to kill it.)

In [ ]:
## load pickle files

